# 3WM gain and TWPAnalysis

This tutorial demonstrates how to utilize the `twpasolver.TWPAnalysis` class to simulate the gain response of a TWPA by solving the Coupled Mode Equations (CMEs).

### Overivew
`twpasolver.TWPAnalysis` automates the procedures required to compute the gain characteristics of a TWPA model.

Implemented features:

* Calculation of phase-matching profile, gain and bandwidth.
* Automatic caching of data.
* Sweeps over lists of input parameters for the analysis functions.
* Simple plots.

Current limitations:

* Specialized for solving 3-wave mixing systems only.
* Simple sytems of CMEs, reflections and additional harmonics not considered.
* Analysis routines are not isolated from the class.
* 1-D sweeps only.

### Example - Setup

Let's start by importing the required libraries and initializing the `TWPAnalysis` instance. This object requires:
* A `TWPA` instance or the name of a json file containing the definition of the model
* A frequency span which determines the full region that will be considered in the analysis, which can be provided either as a list of values or a tuple that will be passed to `numpy.arange`. Some precautions in choosing the frequency span must be taken to correctly unwrap the phase response of the device. Namely, the frequency array should be dense enough and should start from frequencies much lower than the position of the stopband.

In [ ]:
import logging

import matplotlib.pyplot as plt
import numpy as np

from twpasolver import TWPAnalysis
from twpasolver.logging import log
from twpasolver.mathutils import dBm_to_I

log.setLevel(logging.WARNING)

plt.rcParams["font.size"] = 13.5
plt.rcParams["axes.axisbelow"] = True

twpa_file = "model_cpw_dartwars_13nm_Lk8_5.json"
a = TWPAnalysis(twpa=twpa_file, f_arange=(0.5, 10, 0.5e-3))
a.update_base_data()
ax = a.plot_response(pump_freq=a.data["optimal_pump_freq"])
a.twpa.model_dump()

### Phase matching

In [ ]:
_ = a.phase_matching()
a.plot_phase_matching(thin=3)

### Gain and bandwidth

In [ ]:
s_arange = np.arange(1, 7, 0.05)
_ = a.gain(signal_freqs=s_arange, Is0=1e-6)
a.plot_gain()
_ = a.bandwidth()
for edg in a.data["bandwidth"]["bandwidth_edges"]:
    plt.axvline(edg, color="black", ls="--")
plt.axhline(a.data["bandwidth"]["reduced_gain"], c="black", ls="--")
plt.axhline(a.data["bandwidth"]["mean_gain"], c="black", ls="--");

### Parameter sweeps

#### Gain as a function of pump frequency

In [ ]:
optimal = a.data["optimal_pump_freq"]
pumps = np.arange(optimal - 0.2, optimal + 0.5, 0.02)
pumpsweep_res = a.parameter_sweep(
    "gain", "pump", pumps, signal_freqs=s_arange, save_name="pump_profile"
)

In [ ]:
plt.pcolor(pumpsweep_res["signal_freqs"][0], pumps, pumpsweep_res["gain_db"])
plt.xlabel("Signal frequency [GHz]")
plt.ylabel("Pump frequency [GHz]")
c = plt.colorbar()
c.set_label("Gain [dB]")

#### Compression point

In [ ]:
signals_db = np.arange(-80, -40, 1)
signals = dBm_to_I(signals_db)
edges = a.data["bandwidth"]["bandwidth_edges"]
s_arange = a.data["bandwidth"]["bw_freqs"]  # (edges[0], edges[-1], 0.1)
compression_res = a.parameter_sweep(
    "gain", "Is0", signals, signal_freqs=s_arange, thin=1000, save_name="compression"
)

In [ ]:
mean_gains = np.mean(compression_res["gain_db"], axis=1)
reduced_1db = mean_gains[0] - 1
cp_1db = np.interp(reduced_1db, mean_gains[::-1], signals_db[::-1])
plt.scatter(signals_db, mean_gains)
plt.xlabel("Signal power [dB]")
plt.ylabel("Gain [dB]")
plt.axhline(reduced_1db, c="black", ls="--")
plt.axvline(cp_1db, c="black", ls="--")